In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1=pd.read_csv('/kaggle/input/world-happiness/2015.csv')
df2=pd.read_csv('/kaggle/input/world-happiness/2016.csv')

df1.shape,df2.shape


In [ ]:
import dask.dataframe as dd

# df = dd.read_csv('data/2000-*-*.csv')
# df
df=dd.read_csv('/kaggle/input/world-happiness/201*.csv')
df.head()

In [ ]:
csv_file_list = ['/kaggle/input/world-happiness/2015.csv','/kaggle/input/world-happiness/2016.csv']

list_of_dataframes = []
for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename))

merged_df = pd.concat(list_of_dataframes)

 

In [ ]:
merged_df.shape

In [ ]:
df_15 = pd.read_csv('/kaggle/input/world-happiness/2015.csv')
df_15['Year']=np.ones(df_15.shape[0])*2015
df_16 = pd.read_csv('/kaggle/input/world-happiness/2016.csv')
df_16['Year']=np.ones(df_16.shape[0])*2016
df_17 = pd.read_csv('/kaggle/input/world-happiness/2017.csv')
df_17['Year']=np.ones(df_17.shape[0])*2017
df_17=df_17.rename(columns={'Happiness.Rank':'Happiness Rank','Happiness.Score':'Happiness Score',
                           'Economy..GDP.per.Capita.':'Economy (GDP per Capita)',
                           'Health..Life.Expectancy.':'Health (Life Expectancy)',
                           'Trust..Government.Corruption.':'Trust (Government Corruption)',
                           'Dystopia.Residual':'Dystopia Residual'})
df_18 = pd.read_csv('/kaggle/input/world-happiness/2018.csv')
df_18['Year']=np.ones(df_18.shape[0])*2018
df_18=df_18.rename(columns={'Overall rank':'Happiness Rank', 'Country or region':'Country',
                            'Score':'Happiness Score', 'GDP per capita':'Economy (GDP per Capita)',
                            'Healthy life expectancy':'Health (Life Expectancy)',
                            'Freedom to make life choices':'Freedom',
                            'Perceptions of corruption':'Trust (Government Corruption)',
                            'Social support':'Family'})
df_19 = pd.read_csv('/kaggle/input/world-happiness/2019.csv')
df_19['Year']=np.ones(df_19.shape[0])*2019
df_19=df_19.rename(columns={'Overall rank':'Happiness Rank', 'Country or region':'Country',
                            'Score':'Happiness Score', 'GDP per capita':'Economy (GDP per Capita)',
                            'Healthy life expectancy':'Health (Life Expectancy)',
                            'Freedom to make life choices':'Freedom',
                            'Perceptions of corruption':'Trust (Government Corruption)',
                            'Social support':'Family'})

df = pd.concat([df_15,df_16,df_17,df_18,df_19],axis=0, ignore_index=True)
df.drop(columns=['Standard Error','Lower Confidence Interval','Upper Confidence Interval',
                 'Whisker.high','Whisker.low'],inplace=True)

In [ ]:
df.head(4)

In [ ]:
df.isnull().mean()*100

In [ ]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

missing_zero_values_table(df)


In [ ]:
df.describe()

In [ ]:
df.drop(df[(df['Economy (GDP per Capita)']==0) | (df['Family']==0) |(df['Health (Life Expectancy)']==0)].index,inplace=True)

In [ ]:
df 

In [ ]:
df[df['Family']==0]

In [ ]:
 df['Year'] =  df['Year'].astype(int)

In [ ]:
n_data = df.copy(deep=True)

In [ ]:
n_data.drop(['Region'], axis=1,inplace=True)

In [ ]:
n_data

In [ ]:
n_data.fillna(n_data.mean(),inplace=True)
n_data.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
 
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
n_data['Country'] = labelencoder.fit_transform(n_data['Country'])
n_data

In [ ]:
n_data.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(12,8))
sns.heatmap(n_data.corr(),annot=True)
plt.show()

In [ ]:
df.head(1)

In [ ]:
n_data.head()

In [ ]:
X = n_data.iloc[:, n_data.columns != 'Happiness Score']
y = n_data['Happiness Score']

In [ ]:
from sklearn.feature_selection import mutual_info_regression
information = mutual_info_regression(X,y)
value_df = pd.DataFrame({'columns':X.columns.values,'value':information})
value_df.sort_values(by= 'value',ascending = False)

mutual_info_useless = value_df[value_df.value == 0]['columns']

In [ ]:
mutual_info_useless

In [ ]:
import xgboost as xgb
xgbre = xgb.XGBRFRegressor()
xgbre.fit(X, y)
xgb_importance = pd.DataFrame({'columns':X.columns.values,'values':xgbre.feature_importances_}).sort_values(by = 'values',ascending=False)
xgb_useless = xgb_importance[xgb_importance.values==0]['columns']
xgb_useless

In [ ]:
features = n_data.columns

In [ ]:
features[2]

In [ ]:
fig,ax=plt.subplots(11,figsize=(10,80))
for i in range(11):
    sns.boxplot(y=features[i],data=n_data,ax=ax[i])

In [ ]:
df.nlargest(5,columns='Happiness Score')

In [ ]:
n_data.nlargest(5,columns='Happiness Score')

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y ,test_size=.25,random_state=42)
from sklearn.model_selection import cross_val_score,StratifiedKFold
cv = StratifiedKFold(n_splits=10)
def auc_score(model):
    score = cross_val_score(model,X_train,y_train ,cv = 10, n_jobs = -1)
    return(score.mean())
    

In [ ]:
from sklearn.model_selection import cross_validate,KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression,LassoCV, ElasticNet,Lasso
from sklearn.kernel_ridge import KernelRidge

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from imblearn.pipeline import Pipeline # Inorder to avoid testing model on sampled data
from sklearn import model_selection
kf = model_selection.StratifiedKFold(n_splits=5)

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,StandardScaler
from sklearn.svm import SVR
import sklearn
import xgboost as xgb
 
models = {
     'logistic_regression':LinearRegression(),
     'Extra_tree_classifier':sklearn.ensemble.ExtraTreesRegressor(),
     'Decision_tree_classifier':DecisionTreeRegressor(),
     'Random_forest_classifier':sklearn.ensemble.RandomForestRegressor(),
     'Xgboost_classifier': xgb.XGBRegressor(),
     'SVM': SVR(),
     'gbm':sklearn.ensemble.GradientBoostingRegressor(),
     'adaboost':sklearn.ensemble.AdaBoostRegressor() 
}
# for model_names,model in models.items():
#     model_pipeline = make_pipeline(model)
#     print(model_names,auc_score(model_pipeline)
 

In [ ]:
for model_names,model in models.items():
    model_pipeline = make_pipeline(model)
    print(model_names,auc_score(model_pipeline))

In [ ]:
base_ext = make_pipeline(StandardScaler(),sklearn.ensemble.ExtraTreesRegressor(random_state = 42))
auc_score(base_ext)


In [ ]:
base_ext.fit(X_train,y_train)


In [ ]:
y_pre = base_ext.predict(X_test)
 

In [ ]:
r2 = sklearn.metrics.r2_score(y_test,y_pre)
print('R square : ',r2)

In [ ]:
y_pre_t = y_pre = base_ext.predict(X_train)

r2 = sklearn.metrics.r2_score(y_train,y_pre_t)
print('R square : ',r2)

In [ ]:
k=len(X_train.columns)
n = X.size
adj2=1 -((1-r2)*(n-1)/(n-k-1))
print('Adjusted R square : ',adj2 )

In [ ]:
r2-adj2

In [ ]:
y_pre.shape

In [ ]:
print(f"This is testing score : {base_ext.score(a ,y_pre)}")
# print(f"This is training score : {base_ext.score(y_train,base_ext.predict(X_train))}")


In [ ]:
X_train1,X_test1,y_train1,y_test1= train_test_split(X,y ,test_size=.25,random_state=42)


In [ ]:
type(y)

In [ ]:
lr = LinearRegression()
lr.fit(X_train1,y_train1)
y_pred1=lr.predict(X_test1)

In [ ]:
type(y_test)

In [ ]:
type(y_pred1),y_pred1.reshape(192,1).shape

In [ ]:
type(y_test1),y_test1.shape

In [ ]:
type(y_pred1),y_pred1.shape

In [ ]:
pr.shape

In [ ]:
y_test1_val=y_test1.values

In [ ]:
f=y_test1_val 

In [ ]:
f_test=f.reshape(-1,1)

In [ ]:
f_test.shape

In [ ]:
pr=y_pred1.reshape(-1,1)

In [ ]:
pr.shape

In [ ]:
c=y_test1.values

In [ ]:
B = c.reshape(1,  2)

In [ ]:
pr

In [ ]:
lr.score(y_test1,pr) 


In [ ]:
y_test2 = np.reshape(f_test,(1,-1))

In [ ]:
y_pred2 = np.reshape(y_pred1,(1,-1))

In [ ]:
X_train1.shape

In [ ]:
import keras 
from keras.models import Sequential
from keras.layers import Dense,Dropout

cls =Sequential()

cls.add(Dense(units=6,activation='relu',input_dim=10))
cls.add(Dense(4, activation='relu'))
cls.add(Dense(1, activation='sigmoid'))

cls.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:

model_lisr=cls.fit(X_train1,y_train1,validation_split=0.20,batch_size=10,epochs=2)



In [ ]:
y_pred=cls.predict(X_test)

In [ ]:
#from hrere

In [ ]:
df.head(2)

In [ ]:
df9=df.copy()

In [ ]:
df9.drop(['Country'],inplace=True,axis=1)

In [ ]:
df9.head()

In [ ]:
df.Region.unique()

In [ ]:
df9['Region'].fillna(df9['Region'].mode()[0],inplace=True)

In [ ]:
df9['Dystopia Residual'].fillna(df9['Dystopia Residual'].mean(),inplace=True)

In [ ]:
df9.dropna(inplace=True)

In [ ]:
df9.isna().sum()

In [ ]:
df9['region']=labelencoder.fit_transform(df9['Region'])

In [ ]:
df9.head()

In [ ]:
df9

In [ ]:
df9.drop(['Region'],axis=1,inplace=True)

In [ ]:
df9.head()

In [ ]:
X9=df9.drop(["Happiness Score"],axis=1)
y9=df9['Happiness Score']

In [ ]:
X9

In [ ]:
y9

In [ ]:
X_train9,X_test9,y_train9,y_test9= train_test_split(X9,y9 ,test_size=.25,random_state=42)


In [ ]:
lr9 = LinearRegression()
lr9.fit(X_train9,y_train9)
y_pred9=lr9.predict(X_test9)

In [ ]:
y_test9.values.reshape(-1,1).shape